# Reproducing results from article:

In [3]:
# import libraries
# sklearn reference: https://scikit-learn.org/
# pandas reference: https://pandas.pydata.org/
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import metrics
import pandas as pd

In [4]:
# text and numeric classes that use sklearn base libaries
class TextTransformer(BaseEstimator, TransformerMixin):
    """
    Transform text features
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None, *parg, **kwarg):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberTransformer(BaseEstimator, TransformerMixin):
    """
    Transform numeric features
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

In [5]:
# read in your dataframe
df = pd.read_csv(r"C:\Users\klaas.braga\Downloads\AIDI Georgian\AIDI1002 MLP\Final Project\nlp-example\data\DummyDataNLP.csv")

In [6]:
# take a look at the first 5 observations
df.head()

,Confirmed_Test,Confirmed_Recovery,Confirmed_New,Text_Feature,Text_Predictor
0,36.434962,82.983704,34.824456,Blue,Label_1
1,75.349163,81.735731,71.921676,Red,Label_2
2,76.678489,33.588094,14.124835,Yellow,Label_3
3,73.356221,79.441778,56.910324,Orange,Label_1
4,74.451946,59.228624,20.042376,Blue,Label_2


In [7]:
# use the term-frequency inverse document frequency vectorizer to transfrom count of text
# into a weighed matrix of term importance
vec_tdidf = TfidfVectorizer(ngram_range=(1,1), analyzer='word', norm='l2')

In [8]:
# compile both the TextTransformer and TfidfVectorizer 
# to the text 'Text_Feature' 
color_text = Pipeline([
                ('transformer', TextTransformer(key='Text_Feature')),
                ('vectorizer', vec_tdidf)
                ])

In [9]:
# compile the NumberTransformer to 'Confirmed_Test', 'Confirmed_Recovery', 
# and 'Confirmed_New' numeric features
test_numeric = Pipeline([
                ('transformer', NumberTransformer(key='Confirmed_Test')),
                ])
recovery_numeric = Pipeline([
                ('transformer', NumberTransformer(key='Confirmed_Recovery')),
                ])
new_numeric = Pipeline([
                ('transformer', NumberTransformer(key='Confirmed_New')),
                ])

In [10]:
# combine all of the features, text and numeric together
features = FeatureUnion([('Text_Feature', color_text),
                      ('Confirmed_Test', test_numeric),
                      ('Confirmed_Recovery', recovery_numeric),
                      ('Confirmed_New', new_numeric)
                      ])

In [11]:
# create the classfier from RF
clf = RandomForestClassifier()

In [12]:
# unite the features and classfier together
pipe = Pipeline([('features', features),
                 ('clf',clf)
                 ])

In [13]:
# transform the categorical predictor into numeric
predicted_dummies = pd.get_dummies(df['Text_Predictor'])

In [37]:
# split the data into train and test
# isolate the features from the predicted field
text_numeric_features = ['Text_Feature', 'Confirmed_Test', 'Confirmed_Recovery', 'Confirmed_New']
predictor = 'Text_Predictor'

X_train, X_test, y_train, y_test = train_test_split(df[text_numeric_features], df[predictor], 
                                                    test_size=0.25, random_state=42)

In [38]:
# fit the model
pipe.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Text_Feature',
                                                 Pipeline(steps=[('transformer',
                                                                  TextTransformer(key='Text_Feature')),
                                                                 ('vectorizer',
                                                                  TfidfVectorizer())])),
                                                ('Confirmed_Test',
                                                 Pipeline(steps=[('transformer',
                                                                  NumberTransformer(key='Confirmed_Test'))])),
                                                ('Confirmed_Recovery',
                                                 Pipeline(steps=[('transformer',
                                                                  NumberTransformer(key='Confirmed_Recovery'))])),
                                                ('Confirmed_New',
                                                 Pipeline(steps=[('transformer',
                                                                  NumberTransformer(key='Confirmed_New'))]))])),
                ('clf', RandomForestClassifier())])

In [39]:
# predict from the test set
preds = pipe.predict(X_test)

In [40]:
# see how you did, since this is randomly generated data,
# I would say this accuracy is pretty good :D
print("Accuracy:",metrics.accuracy_score(y_test, preds))

Accuracy: 0.4473684210526316
